In [18]:
import os
from dotenv import load_dotenv

load_dotenv()
huggingfacehub_api_key= os.getenv('HUGGINGFACEHUB_API_TOKEN')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [19]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re
from langchain import OpenAI, LLMChain
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool
from langchain.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun 
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from typing import List, Union
from langchain.agents.agent_toolkits import FileManagementToolkit
from tempfile import TemporaryDirectory
from langchain.tools import ShellTool
from langchain.agents import load_tools
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [20]:
# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()
#Shell Tool
shell_tool = ShellTool()
requests_tools = load_tools(["requests_all"])

#DuckDuckGo Class
duckduck_search = DuckDuckGoSearchRun()

#Python REPL Class
python_repl = PythonREPL()

shell_tool = Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
)


#DuckDuckGo Tool
duckduck_tool=Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
)

#Python REPL Agent Tool Class
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run
)
toolkit = FileManagementToolkit(root_dir=str(working_directory.name)) # If you don't provide a root_dir, operations will default to the current working directory
directory_tools = toolkit.get_tools()

In [21]:

tools = [
    WriteFileTool(),
    ReadFileTool(),
    repl_tool,
    duckduck_tool, 
    #directory_tools, 
    shell_tool, 
    #requests_tool
]

In [22]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

In [23]:
docs = [
    Document(page_content=t.description, metadata={"index": i})
    for i, t in enumerate(tools)
]

In [24]:
vector_store = FAISS.from_documents(docs, OpenAIEmbeddings())

In [25]:
retriever = vector_store.as_retriever()


def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [tools[d.metadata["index"]] for d in docs]

In [26]:
# Set up the base template
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to be as creative as possible!

Question: {input}
{agent_scratchpad}"""

In [27]:
from typing import Callable


# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [28]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

In [29]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

In [30]:
output_parser = CustomOutputParser()

In [31]:
#llm = OpenAI(temperature=0)

In [34]:
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# There are many CallbackHandlers supported, such as
# from langchain.callbacks.streamlit import StreamlitCallbackHandler

callbacks = [StreamingStdOutCallbackHandler()]
gpt4_llm = GPT4All(model="../../../../Models/LLMs/GPT4All/wizardlm-13b-v1.1-superhot-8k.ggmlv3.q4_0.bin", verbose=True)



Found model file at  ../../models/ggml-gpt4all-j-v1.3-groovy.bin
gptj_model_load: loading model from '../../models/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [36]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=gpt4_llm, prompt=prompt, verbose=True)

In [37]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)

In [38]:
def _handle_error(error) -> str:
    return str(error)[:50]

In [41]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=_handle_error,
)

In [42]:
agent_executor.run("What is the quantum gravity?'", debug=True)

AttributeError: module 'langchain' has no attribute 'debug'